In [2]:
from IPython.display import display, Image
from IPython.display import HTML
import numpy as np

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

神经网络模型一般依靠随机梯度下降进行模型训练和参数更新，网络的最终性能与收敛得到的最优解直接相关，而收敛效果实际上有很大程度取决于网络参数的初始化。糟糕的初始化不仅会影响网络收敛，甚至会导致"梯度弥散"或"梯度爆炸"。

### 全零初始化
通过合理的数据预处理和规范化，当网络收敛到稳定状态时，参数（权值）在理想情况下应基本保持正负各半的状态（此时期望为0）。因此，一种简单且听起来合理的参数初始化做法是，将所有参数都初始化为0，因为这样可使得初始化全零时参数的期望与网络稳定时参数的期望一致为零。但是参数全为0时网络不同神经元的输出必然相同，相同输出则导致梯度更新完全一样，这样便会令更新后的参数仍然保持一样的状态。换句话说，如若参数进行了全零初始化，那么网络神经元将无法更新参数，从而无法进行模型训练。

### 随机初始化
将参数值随机设定为接近于0的一个很小的随机数（有正有负）。在实际应用中，随机参数服从高斯分布或均匀分布都是较有效的初始化方式。

In [5]:
# n_in: 输入神经元个数
n_in = 100
# n_out: 输出神经元个数
n_out = 10
# 0.001为控制参数纲的因子，使参数期望能保持在接近0的较小数值范围内
w = 0.001 * np.random.randn(n_in, n_out)

但是，上述做法仍会带来一个问题，即网络的输出数据分布的方差会随着输入神经元个数改变。为解决这一问题，会在初始化的同时加上对方差大小的规范化，如除以n的平方根，n为输入神经元个数，有时也可指定为(n_in + n_out) / 2
$$
Var(s)\\
= Var(\sum_{i}^{n} w_{i} x_{i})\\
=\sum_{i}^{n}Var(w_{i} x_{i}) \\
=\sum_{i}^{n}E[(w_{i} x_{i} - E[w_{i} x_{i}]^{2})]\\
=\sum_{i}^{n}(E[(w_{i} x_{i})^{2}] - (E[w_{i} x_{i}])^{2})\\
=\sum_{i}^{n}(E[w_{i}^{2}x_{i}^{2}] - (E[w_{i}]E[x_{i}])^{2})\\
=\sum_{i}^{n}(E[w_{i}^{2}]E[x_{i}^{2}] - (E[w_{i}])^2(E[x_{i}])^{2})\\
=\sum_{i}^{n}((E[w_{i}^{2}]-(E[w_{i}])^2+(E[w_{i}])^2)(E[x_{i}^{2}]-(E[x_{i}])^2+(E[x_{i}])^2) - (E[w_{i}])^2(E[x_{i}])^{2})\\
=\sum_{i}^{n}((Var(w_{i})+(E[w_{i}])^2)(Var(x_{i})+(E[x_{i}])^2) - (E[w_{i}])^2(E[x_{i}])^{2})\\
=\sum_{i}^{n}(Var(w_{i})Var(x_{i})+Var(w_{i})(E[x_{i}])^2+Var(x_{i})(E[w_{i}])^2+(E[w_{i}])^2(E[x_{i}])^2 - (E[w_{i}])^2(E[x_{i}])^{2})\\
=\sum_{i}^{n}(E[w_{i}])^{2}Var(x_{i}) + (E[w_{i}])^{2}Var(w_{i}) + Var(x_{i})Var(w_{i})\\
=\sum_{i}^{n} 0 * Var(x_{i})+ 0 * Var(w_{i})  + Var(x_{i})Var(w_{i})\\
=(nVar(w))Var(x) $$

输入s经过非线性变换，即s=∑w(i)x(i)，又因x各维服从独立同分布的假设，可推出∑w(i)x(i) => ∑Var(w(i)x(i))，通过方差公式展开。理想情况下，处于稳定状态的神经网络参数和数据均值应为0，所以上述公式中的E[wi]=E[xi]=0，最终得到(nVar(w))Var(x)。为了保证输入数据Var(s)和输出数据Var(x)方差一致：$$Var(s) = (nVar(w))Var(x)$$

所以令：$$nVar(w)=1\\
n\cdot Var(w)=n\cdot Var(a{w}')=n\cdot a\cdot Var({w}')=1$$

其中w′为方差规范化后的参数，则：$$a=\sqrt{(1/n)}$$

使用Xavier方法保持了输入输出数据分布方差的一致性。

In [6]:
# Xavier 参数初始化方法
n = 100
w = (0.001 * np.random.randn(n_in, n_out)) / np.sqrt(n)

但是Xavier的缺点是没有考虑到非线性映射函数对输入s的影响，因为使用如ReLU函数等非线性映射函数后，输出数据的期望往往不再为0，即E[xi]!=0。一种改进方法He参数初始化方法是将非线性映射造成的影响考虑进参数初始化中，Xavier方法中方差规范化的分母应为根号(n/2)，而不是根号n。

In [ ]:
# He参数初始化方法
n = 100
w = (0.001 * np.random.randn(n_in, n_out)) / np.sqrt(n/2)

以上是参数初始化分布服从高斯分布的情形，均匀分布也是一种很好的初始化分布，当参数初始化服从均匀分布，由于分布性质的不同，均匀分布需要指定其取值区间。

In [ ]:
# 均匀分布的Xavier
# 均匀分布的区间[low, high]
low = -np.sqrt(3/n)
high = np.sqrt(3/n)
# n_in: 输入神经元个数
n_in = 100
# n_out: 输出神经元个数
n_out = 10
rand_param = a + (b-a) * np.random.randn(n_in, n_out)
w = 0.001 * rand_param

In [ ]:
# 均匀分布的He
# 均匀分布的区间[low, high]
low = -np.sqrt(6/n)
high = np.sqrt(6/n)
# n_in: 输入神经元个数
n_in = 100
# n_out: 输出神经元个数
n_out = 10
rand_param = a + (b-a) * np.random.randn(n_in, n_out)
w = 0.001 * rand_param

### 其它初始化方法
除了直接随机初始化网络参数，一种简便易行且十分有效的方法是利用预训练模型(pre-trained model)，将预训练模型的参数作为新任务上模型的参数初始化。由于预训练模型已经在原先任务(如ImageNet、Places205等数据集)上收敛到较理想的局部最优解，加上很容易获得这些预训练模型，用此最优解作为新模型的参数初始化。

### 总结
- 网络参数初始化的好坏很大程度影响了网络模型的最终效果；
- 推荐的网络初始化方法为He方法，将参数初始化为服从高斯分布或均匀分布的较小随机数，同时对参数方差需实行规范化；
- 使用预训练模型中的参数作为新模型的参数初始化，是一种有效的参数初始化方法。